<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
</font>
</h1>
<h1 align=center>
 SLIC-segmentation
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
توجه 1 : در این نوت بوک از فونت vazir برای بیان توضیحات استفاده شده است ، که لینک نصب این فونت در ادامه آمده است.

[vazir font download link](https://quera.ir/qbox/view/jLCUyovUmo/vazir-font-v30.1.0.zip)
</font>
</p>

<hr>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در ابتدا کتابخانه های موردنیاز را لود می کنیم 
</font>
</p>

In [1]:
# import libraries:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import morphology

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تابع <code> show </code> را برای نمایش تصاویر حاصل در مراحل مختلف کد و برای بررسی نتایج استفاده می کنیم
</font>
</p>

In [ ]:
def show(img,height=500):
    img1 = img.copy()
    h1, w1 = img1.shape[0], img1.shape[1]
    scale1 = height / h1
    dim1 = (int(scale1*w1), int(scale1*h1))
    resized_img1 = cv2.resize(img1, dim1, interpolation = cv2.INTER_AREA)
    cv2.imshow('image', resized_img1)
    cv2.waitKey(0)  
    cv2.destroyAllWindows()

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در تابع <code>find_gradient</code> که از آن در تابع اصلی <code>make_slic</code> استفاده شده است :
<br>
تصویر gray را ورودی گرفته و با استفاده از دو کرنل در راستای افقی و عمودی مقدار اندازه ی گرادیان هر پیکلسل تصویر ورودی را به دست آورده(با استفاده از تابع np.hypot) و ماتریس حاصل را با نام 
gradient_image خروجی می دهیم.
</font>
</p>

In [ ]:
def find_gradient(gray_img):
    x_kernel = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
    y_kernel = x_kernel.T
    x_gradient = cv2.filter2D(gray_img, -1, x_kernel)
    y_gradient = cv2.filter2D(gray_img, -1, y_kernel)
    gradient_img = np.hypot(x_gradient, y_gradient)

    return gradient_img

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
از تابع <code>find_distributed_centroids</code> برای یافتن نقاط اولیه با توزیع منظم در تصویر img استفاده می شود با این ویژگی که این نقاط اولیه را در یک پنجره 5*5 حرکت داده تا کمترین مقدار گرادیان را نیز در آن پنجره داشته باشد این کار با استفاده از ورودی gradient_img انجام میگیرد و هم چنین مقدار ورودی k تعداد نقاط ما را مشخص می کنند:
<br>
در ابتدا ، با توجه به مساحت ماتریس img و مقدار k نقاط اولیه ی موردنظر برای مراکز خوشه ها را به دست می آوریم؛ سپس به ازای هر مرکز خوشه ، یک پنجره ی 5*5 حول آن در نظر گرفته و با توجه به ماتریس gradient_img آن مرکز خوشه ی اولیه را به پیکسلی که در آن پنجره کمترین مقدار گرادیان دارد ، منتقل می کنیم و در نهایت مراکز خوشه های نهایی را به همراه مقدار s (فاصله ی دو خوشه ی متوالی) خروجی می دهیم.
</font>
</p>

In [ ]:
def find_distributed_centroids(img, gradient_img, k):
    img_area = img.shape[0] * img.shape[1]
    s = np.sqrt(img_area // k)  # s is difference between two cluster in initial state
    init_x_centroid = np.arange(s//2, img.shape[0], s)
    init_x_centroid = np.uint32(init_x_centroid)

    init_y_centroid = np.arange(s//2, img.shape[1], s)
    init_y_centroid = np.uint32(init_y_centroid)

    # join x_centroid and y_centroid:
    init_centroid = []
    for i in range(init_x_centroid.shape[0]):
        for j in range(init_y_centroid.shape[0]):
            init_centroid.append([init_x_centroid[i], init_y_centroid[j]])
    init_centroid = np.array(init_centroid)

    # move centroid to the points with lowest gradint with 5*5 window arount
    # each cluster center:
    centroid = []
    for i in range(init_centroid.shape[0]):
        init_x_c = init_centroid[i, 0]
        init_y_c = init_centroid[i, 1]
        window_gradient = gradient_img[init_x_c-2:init_x_c+3, init_y_c-2:init_y_c+3]
        min_inds = np.where(window_gradient == np.amin(window_gradient))
        rand_ind = np.random.randint(min_inds[0].shape)   # for find pixel with minimum gradient randomly
        
        # find x_c and y_c and append to centroid
        x_c = min_inds[0][rand_ind] + init_x_c-2
        y_c = min_inds[1][rand_ind] + init_y_c-2
        centroid.append([x_c[0], y_c[0]])

    centroid = np.array(centroid)

    return (s, centroid)

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تابع <code>make_slic</code> : که تصویر اصلی و مقادیر k و alpha را گرفته و تصویر oversegment شده را خروجی می دهد.
<br>
در ابتدا برای اینکه حجم و مدت زمان محاسبات زیاد نشود ، تصویر را با نسبت 0.6 ، resize می کنیم و سایر عملیات را روی تصویر کوچک شده اعمال می کنیم.
<br>
در ادامه تصویر gray و تصویر در فضای LAB متناظر با تصویر اصلی به دست می آوریم و با استفاده از تصویر خاکستری ماتریس gradient_img و با استفاده از تابع <code>find_distributed_centroids</code> مختصات مراکز خوشه ی منظم را به دست می آوریم و در ادامه ی کد در یک حلقه ی همواره true به صورت زیر مختصات مراکز خوشه ها را تغییر داده تا به یک oversegmentation مناسب دست پیدا کینم.
<br>
در هر حلقه داریم :
<br>
ابتدا دو ماتریس dist_img و label_img را مقدار دهی اولیه می کنیم تا در ادامه ی کار به درایه های مختلف آن مقدار بدهیم، هر درایه ی ماتریس dist_img مقدار فاصله ی پیکسل متناظر تا نزدیکترین مرکز خوشه را در خود نگه می دارد و در ابتدا هم این ماتریس با مقدار بی نهایت مقدار دهی اولیه شده است.
هم چنین هر درایه ی ماتریس label_img نشاندهنده ی لیبل پیکسل متناظر می باشد که در ابتدا با صفر مقدار دهی شده است.
<br>
در ادامه به ازای هر مرکز خوشه موجود در ماتریس centroid ، پنجره ی 5*5 حول آن را با استفاده از ماتریس lab_img (تصویر در فضای LAB) به دست آورده و هم چنین مقدار x و y هر پیکسل پنجره ی مذکور را به دست می آوریم و با اضافه کردن این دو پنجره ی x و y به پنجره ی در فضای LAB یک پنجره ی با بعد 5 را تشکیل می دهیم و با نام <code>feature_window</code> را به دست می آوریم.
<br>
در ادامه ی کار ، هر یک از 5 بعد پنجره را به صورت یک بردار تک بعدی کرده و در عرض به هم می چسبانیم تا به ماتریس feature_matrix که یک ماتریس دو بعدی با عرض 5 می باشد ، برسیم.
<br>
از سوی دیگر بردار 1*5 موسوم به <code>centroid_vector</code> را که در واقع بردار ویژگی مرکز خوشه ی i ام می باشد را در نظر می گیریم.
با استفاده از سه ستون اول هر یک از ماتریس های feature_matrix و centroid_vector فاصله در فضای lab را به ازای تمام پیکسل های پنجره ی 5*5 تا مرکز خوشه ی متناظر به دست می آوریم و ماتریس d_lab_matrix را به دست می آوریم ؛ هم چنین با استفاده از دو ستون آخر ، ماتریس d_xy_matrix را به دست می آوریم و با استفاده از مقدار alpha و ترکیب دو ماتریس به دست آمده مذکور ، ماترس d_matrix را به دست می آوریم که فاصله ی نهایی هر پیکسل در پنجره را تا مرکز خوشه ی متناظر نشان می دهد.
<br>
در بخش بعدی پنجره ی متناظر را از دو ماتریس dist_img و label_img در نظر گرفته و با نام های dist_window و label_window در نظر می گیریم ، حال به ازای هر درایه ی درون دو پنجره اگر که مقدار فاصله ی نهایی پیکسل مذکور در حلقه ی کنونی از مقدار فاصله ای که تا به حل برایش ثبت شده مقدار فاصله را برای آن پیکسل به روزرسانی کرده و هم چنین لیبل آن را به مقدار خوشه ی i تغییر می دهیم.
<br>
و سپس دو ماتریس اصلی dist_img و label_img را با استفاده از پنجره های آپدیت شده، به روز می کنیم.
<br>
فرآیند فوق را برای تمام مراکز خوشه انجام میدهیم و پس از این که حلقه روی مراکز خوشه به اتمام رسید ، مراکز خوشه را با استفاده از لیبل های جدید ماتریس های label_img به دست آورده و نام آن را <code>new_centroid</code> قرار می دهیم.
و فاصله ی مراکز قبلی (تحت عنوان centroid) با مراکز جدید را به دست می آوریم.
<br>
شرط پایان الگوریتم را یکی از دو حالت زیر در نظر می گیریم:
<br>
1. اگر که حداکثر اختلاف بی دو مرکز قدیم و جدید کوچکتر از 30 شود.
<br>
2. درصورتی که تعداد iteration هایی که تمام این مراحل به ازای تمام مراکز خوشه انجام می شود ، بیشتر از 10 شود .
(این شرط به جهت جلوگیری از طولانی شدن زمان اجرای الگوریتم لحاظ شده و با توجه به بررسی ها ، در حدود 5 iteration به یک خوشه بندی مناسبی می رسیم.)
این نکته نیز ذکر می شود که در اغلب تست ها با شرط اول از حلقه ی اولیه خارج می شویم.
<br>
درنهایت نیز ، پس از اینکه لیبیل های نهایی هر پیکسل مشخص شد ، با استفاده از روش های morphology و به طور خاص closing ، عملیات enforce connectivity را برای اینکه از تعدد زیاد قطعات بسیار کوچک تشکیل شده جلوگیری کنیم ، استفاده میکنیم.
<br>
سپس با استفاده از laplacian مرزهای قطعات را بر اساس لیبل ها به دست می آوریم و این مرزبندی در هر سه کانال اعمال کرده و در پایان این تابع تصویر oversegment شده با استفاده از روش slic خروجی داده می شود.
</font>
</p>

In [ ]:
def make_slic(img, k , alpha):

    # Resize the image to spend less time
    scale_percent = 0.6
    w = int(img.shape[1] * scale_percent )
    h = int(img.shape[0] * scale_percent )
    img = cv2.resize(img, (w, h))

    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # find gradient of gray_img:
    gradient_img = find_gradient(gray_img)

    # start slic algorithm:
    # first using find_distributed_centroids finds centroid
    (s, centroid) = find_distributed_centroids(img, gradient_img, k)

    count = 0
    while(1):
        # produce a numpy array with shape = img that keep minimum cost for each img pixel
        # that assigned to its cluster center.
        # initially with inf value
        # and also produce label_img that keep label for each pixel initial with nan
        dist_img = np.full((img.shape[0], img.shape[1]), np.inf)
        label_img = np.zeros((img.shape[0], img.shape[1]))

        
        for i in range(centroid.shape[0]):
            ############# find feature space with (l,a,b,x,y) for each pixel #############
            x_c = centroid[i, 0]
            y_c = centroid[i, 1]

            x_min_range = max(0, x_c - int(s))
            x_max_range = min(img.shape[0], x_c + int(s))
            y_min_range = max(0, y_c - int(s))
            y_max_range = min(img.shape[1], y_c + int(s))

            lab_window = lab_img[x_min_range:x_max_range, y_min_range:y_max_range]
            x_range = np.arange(x_min_range, x_max_range)
            y_range = np.arange(y_min_range, y_max_range)
            x_window = np.transpose([x_range] * lab_window.shape[1])
            y_window = np.tile(y_range, (lab_window.shape[0], 1))

            # combine lab_window, x_window and y_window:
            feature_window = np.zeros((lab_window.shape[0], lab_window.shape[1], 5))
            feature_window[:, :, 0:3] = lab_window
            feature_window[:, :, 3] = x_window
            feature_window[:, :, 4] = y_window

            # convert 3d array feature_window to 2d feature_matrix:
            l_vector = feature_window[:, :, 0].flatten()
            a_vector = feature_window[:, :, 1].flatten()
            b_vector = feature_window[:, :, 2].flatten()
            x_vector = feature_window[:, :, 3].flatten()
            y_vector = feature_window[:, :, 4].flatten()

            feature_matrix = np.zeros((l_vector.shape[0], 5))
            feature_matrix[:, 0] = l_vector  # first column of feature_matrix is l_vector
            feature_matrix[:, 1] = a_vector
            feature_matrix[:, 2] = b_vector
            feature_matrix[:, 3] = x_vector
            feature_matrix[:, 4] = y_vector  # last column of feature_matrix is y_vector

            
            ############# find distance for each feature vector from i'th centroid #############
            centroid_vector = np.array([lab_img[x_c, y_c,0],
                                        lab_img[x_c, y_c,1],
                                        lab_img[x_c, y_c,2],
                                        x_c,
                                        y_c])

            d_lab_matrix = np.sum((feature_matrix[:,0:3] - centroid_vector[0:3])**2, axis=1)
            d_xy_matrix = np.sum((feature_matrix[:,3:5] - centroid_vector[3:5])**2, axis=1)
            d_matrix = d_lab_matrix + alpha*d_xy_matrix

            ############# convert d_matrix to d_window for comparing with dist_img and assign label  #############
            dist_window = dist_img[x_min_range:x_max_range, y_min_range:y_max_range].copy()
            label_window = label_img[x_min_range:x_max_range, y_min_range:y_max_range].copy()
            d_window = d_matrix.reshape(x_range.shape[0], y_range.shape[0])

            label_window[dist_window > d_window] = i
            dist_window[dist_window > d_window] = d_window[dist_window > d_window]
            
            # update dist_img and label_img:
            dist_img[x_min_range:x_max_range, y_min_range:y_max_range] = dist_window.copy()
            label_img[x_min_range:x_max_range, y_min_range:y_max_range] = label_window.copy()

        ############# find new centroids  #############
        new_centroid = []
        for i in range(centroid.shape[0]):
            new_centroid_inds = np.where(label_img == i)
            new_x_c = int(np.mean(new_centroid_inds[0]))
            new_y_c = int(np.mean(new_centroid_inds[1]))
            new_centroid.append([new_x_c, new_y_c])

        new_centroid = np.array(new_centroid)


        ############# compute difference between new_centroid and centroid(previous centers) #############
        centroid_diff_tmp = centroid - new_centroid
        centroid_diff = np.hypot(centroid_diff_tmp[:,0], centroid_diff_tmp[:,1])

        if (np.amax(centroid_diff) < 30  or count > 10):
            break
        else:
            count += 1
            print(f'differ = {np.amax(centroid_diff)}')
            print(f'count = {count}')
            centroid = new_centroid


    ################# after finding label_img(label for each cluster) we have: ###############
    # using closing from morphology to Enforce connectivity:
    label_closing = morphology.closing(label_img, np.ones((30, 30), np.uint8))

    # from label_closing and using lapalcian find boundaries and apply to image
    label_boundaries = np.uint8(cv2.Laplacian(label_closing, -1, ksize=3)) > 1
    img_boundaries = cv2.merge([1-label_boundaries,1-label_boundaries,1-label_boundaries])
    slic_img = img*img_boundaries

    return slic_img   

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در بدنه ی کد ، ابتدا تصویر slic.img را لود می کنیم و به ازای مقادیر مختلف k که در صورت سوال ذکر شده ، تصاویر oversegment شده را تولید کرده و ذخیره می نماییم.
لازم به ذکر است که با توجه به بررسی های انجام شده مقدار alpha را برابر با 0.3 در نظر می گیریم تا نتایج نسبتا مناسبی حاصل شوند .
در واقع مقدار معقول برای alpha بایستی در بازه ی 0.2 تا نیم باشد چرا که نزدیک بودن alpha به صفر باعث بی تاثیر شدن مختصات پیکسل ها و بیش از نیم بودن این مقدار موجب تاثیر بیش از حد مختصات و در نتیجه خوشه بندی بسیار منظم می شود که مطلوب ما نیست لذا در نهایت مقدار alpha را برابر با 0.3 در نظر می گیریم.
</font>
</p>

In [ ]:
img = cv2.imread('org_img.jpg')
slic_64 = make_slic(img, 64, 0.3)
slic_256 = make_slic(img, 256, 0.3)
slic_1024 = make_slic(img, 1024, 0.3)
slic_2048 = make_slic(img, 2048, 0.3)

In [ ]:
cv2.imwrite('result_imgs/slic_64_segment.jpg', slic_64)
cv2.imwrite('result_imgs/slic_256_segment.jpg', slic_256)
cv2.imwrite('result_imgs/slic_1024_segment.jpg', slic_1024)
cv2.imwrite('result_imgs/slic_2048_segment.jpg', slic_2048)

True